In [1]:
import json
import numpy as np
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/all.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

total words: 47479434


In [5]:
stopwords = set(one_time)

In [6]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Первая неделя марта 2015

In [7]:
mart_start = 1425157200000
mart_end = 1425762000000

In [8]:
mart_news = list(filter(lambda x: x.date > mart_start and x.date < mart_end, news))
mart_content = [filter_words(x.content) for x in mart_news]
print("count mart news: %s" % len(mart_content))

count mart news: 625


In [9]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)
tfidf_matrix = tfidf_vectorizer.fit_transform(mart_content)
print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))

vocabulary size: 18525


## DBScan

In [10]:
from sklearn.cluster import DBSCAN

In [11]:
# X = (tfidf_matrix * tfidf_matrix.T).A

In [12]:
db = DBSCAN(eps=1.3, min_samples=3).fit(tfidf_matrix)
# 1.36 for (1, 3)
# 1.12 for 1

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 19
-1: 183, 0: 3


In [13]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 183, titles:
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	В России могут появиться продуктовые карточки для льготников и бедных
	Сумерки — Царского Села под Петербургом разукрасили огни светового шоу
	Американского агента Томаса выдворили из России
	Пореченков назвал слухи о ссоре с "Беларусьфильм" ложью
	Ребенок из России назван самым трогательным суперзлодеем Интернета
	В России вспоминают подвиг Псковских десантников
	Где летом в Москве будут самые длинные пробки
	Футболисты «Челси» выиграли Кубок английской Лиги
	Одного из первых террористов СССР до сих пор содержат в кубанской спецклинике
	Турфирмы зафиксировали спад по всем массовым направлениям
	Сергей Пугачев отказался от адвокатов из-за нехватки средств
	Вся Британия обсуждает ролик, на котором из окна Букингемского дворца вылезает… голый мужчина
	Участники силовой операции записали видеопослание 

# DBScan with Normalizer

In [42]:
normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(tfidf_matrix)
db = DBSCAN(eps=1.2, min_samples=3).fit(X)
labels = db.labels_
print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 23
-1: 352, 0: 151


In [43]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 352, titles:
	Американский астронавт почтил память Леонарда Нимоя в космосе
	Власти Севастополя национализировали завод Порошенко
	Что изменилось с 1 марта: Кредиты - в хорошие руки, лекарства - под госконтроль
	Лондон заблокировал сделку по покупке фондом Фридмана «дочки» RWE
	В Севастополе национализирован завод Порошенко
	Путеводитель по удивительным местам России: три самые необычные пещеры
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Два кота в одном доме: как сделать из них дружную банду?
	Турецкий суд обязал мужчину заплатить жене за нелюбовь
	Хью Джекман покатался на велосипеде по Красной площади и рассказал о своих "секретах красоты"
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	В России могут появиться продуктовые карточки для льготников и бедных
	Сумерки — Царского Села под Петербургом разукрасили огни светового шоу
	Американского агента Томаса выдворили из России
	Российские саночники зав

# DBScan with LSA

In [14]:
svd = TruncatedSVD(50)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

In [15]:
X = lsa.fit_transform(tfidf_matrix)
db = DBSCAN(eps=0.6, min_samples=3).fit(X)
labels = db.labels_
print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))
print("-1: %s, 0: %s" % (labels.tolist().count(-1), labels.tolist().count(0)))

count clusters: 44
-1: 281, 0: 112


In [16]:
print_clusters(mart_news, labels)

Cluster: -1, count news: 281, titles:
	Американский астронавт почтил память Леонарда Нимоя в космосе
	Что изменилось с 1 марта: Кредиты - в хорошие руки, лекарства - под госконтроль
	Лондон заблокировал сделку по покупке фондом Фридмана «дочки» RWE
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Два кота в одном доме: как сделать из них дружную банду?
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	В России могут появиться продуктовые карточки для льготников и бедных
	Сумерки — Царского Села под Петербургом разукрасили огни светового шоу
	Американского агента Томаса выдворили из России
	Пореченков назвал слухи о ссоре с "Беларусьфильм" ложью
	США могут поменять Порошенко на Яценюка
	В России вспоминают подвиг Псковских десантников
	Киевская коалиция трещит по швам, Украина - на пороге дефолта
	Для Порошенко князь Владимир - основатель Украины, а для бандеровцев - русский захватчик
	Дебальцево возвращается к жизни
	Байда и провока

In [17]:
print_topics(svd.components_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
немцов убийство борис дурицкая мост москва анна россия человек дело преступление мочь москворецкий политик украинский украина следствие политика февраль центр
Topic #1:
украина россия российский президент компания украинский цена губернатор хорошавин шахта страна рубль помощь правительство савченко санкция взрыв проект человек становиться
Topic #2:
шахта взрыв засядько горняк донецк украина украинский помощь гуманитарный шахтер погибший пострадать колонна донецкий савченко авария погибать семья метан спасатель
Topic #3:
губернатор хорошавин сахалинский область сахалин александр президент глава задерживать савченко задержание следственный взятка обыск подозреваемый регион правительство дадаев дело арест
Topic #4:
савченко украинский украина российский надежда порошенко голодовка летчица президент гончаренко сизо письмо путин депутат решение заявлять граница журналист вгтрк пересечение
Topic #5:
цена нефть доллар рубль курс шахта торги биржа компания баррель россия рынок немцов

# KMeans

In [18]:
from sklearn.cluster import KMeans

In [19]:
num_clusters = 50

km = KMeans(n_clusters=num_clusters, max_iter=100, n_init=10, n_jobs=-1).fit(tfidf_matrix)

clusters = km.cluster_centers_
labels = km.labels_
print_topics(km.cluster_centers_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
зарплата президент сокращение указ правительство сокращать совет администрация фскн федерация федеральный россия депутат государственный глава госдума путин экземпляр служащий аппарат
Topic #1:
немцов убийство дадаев борис губашев задерживать подозреваемый анзор заур задержание бортников преступление кавказ дело александр дурицкая двое исполнитель анна следствие
Topic #2:
немцов убийство дурицкая борис анна мост девушка следствие автомобиль машина следственный москворецкий москва убийца дело версия преступление камера украинский февраль
Topic #3:
шахта взрыв засядько горняк донецк донецкий шахтер погибший земля украина метан предприятие происходить республика погибать человек народный авария метановый пострадать
Topic #4:
компания мечесть мечело минеев долг сделка млрд британский бард подавать фэшн банк пресс дюймовый производство банкротство снижение арбитражный камень экран
Topic #5:
ребенок школа семья родитель мама девочка люцифер говорить малыш мужчина самый наталья стан

In [20]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 10, titles:
	Исследование РБК: какие газеты и журналы читают чиновники
	Шувалов займется финансовой стабильностью вместо Силуанова
	Кремль посоветовал депутатам просить о снижении зарплат правительство
	Минфин подготовил указ о сокращении 10% федеральных госслужащих
	Исаев рассказал о мучениях топ-менеджеров госкорпораций из-за больших зарплат
	Путин урезал зарплату себе и Медведеву на 10%
	Путин сократил зарплаты, начиная с себя
	Виктор Иванов поставил точку в истории с ликвидацией ФСКН
	Путин уменьшил зарплату себе и Медведеву
	На 10% сокращаются зарплаты Президента, Премьер-министра, глав Прокуратуры и Следственного комитета
Cluster: 1, count news: 22, titles:
	В Москве задержаны подозреваемые в убийстве Бориса Немцова
	Директор ФСБ сообщил о задержании подозреваемых в убийстве Немцова
	Бортников: задержаны подозреваемые в убийстве Немцова
	Названы имена подозреваемых в убийстве Немцова
	Задержаны подозреваемые в убийстве Бориса Немцова
	ФСБ задержала двух по

# KMeans with LSA

In [21]:
svd = TruncatedSVD(70)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(tfidf_matrix)
km = KMeans(n_clusters=num_clusters, max_iter=100, n_init=10, n_jobs=-1).fit(X)
labels = km.labels_

print_topics(km.cluster_centers_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
абан абстракция аварийный абакан абиссинский абориген авиаперевозка авиалиния аббревиатура австриец авиасимулятор авиакомпания августа авиация абрамов аванс абзац авария авен абрам
Topic #1:
абакан абдурахим абсолютно абсурд абрамов абинский абрам аваш абстрактный авантюрист авиаотрасль абрамыч авиатор аварийность авиаперевозчик августа аваков абориген австриец аватар
Topic #2:
абакан абдурахим абориген абрамович аборт абдулатипова абхазия авиалиния авиабаза аварийность авральный авиация аварец авиаузел авиаотрасль абдулов авиаперевозка авиапарк аванс авиалюбитель
Topic #3:
абан аварский абакан авиабаза аватара авиакорпуса австралийский авантюрист аваков авилон аварец абстрактный авиационный абрамыч абрамович авиастроительный аборт абсурд абхазский авиаузел
Topic #4:
абакан августа аварийность абан авиаузел аббревиатура абсолютный авдеевка авиалиния авиапарк абрам абрамыч абстракция аваков абхазия авиакатастрофа аварский авиация авиалюбитель авиатор
Topic #5:
абан абакан авар

In [22]:
print_clusters(mart_news, labels)

Cluster: 0, count news: 10, titles:
	Власти Севастополя национализировали завод Порошенко
	В Севастополе национализирован завод Порошенко
	Мощный пожар вспыхнул ночью в Пенсильвания на востоке США
	Донецк зафиксировал 25 нарушений режима тишины
	Быстрее, выше, сильнее: стереотипы транспортного строительства в России
	«Елочки» и выделенки для велосипедистов
	В Севастополе день вхождения в состав России объявлен выходным
	Басурин: трудно сказать, действительно ли Киев отводит войска
	ДНР отводит минометы от линии соприкосновения по собственной инициативе
	Экономика Мариуполя трещит по швам: посевная кампания под угрозой срыва
Cluster: 1, count news: 10, titles:
	Из-за траурного марша движение в центре Москвы ограничат
	Крупнейшие бизнесмены не пойдут на шествие памяти Бориса Немцова
	В центре Москвы завершился марш памяти Бориса Немцова
	Марш в память о Немцове: в Москве усилена безопасность
	В центре Москвы начался марш памяти Бориса Немцова
	Траурный марш памяти Бориса Немцова: онлайн-

# LSA

In [23]:
lsa = TruncatedSVD(n_components=50, n_iter=100)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [24]:
print_topics(lsa.components_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
немцов убийство борис дурицкая мост москва анна россия человек дело преступление мочь москворецкий политик украинский украина следствие политика февраль центр
Topic #1:
украина россия российский президент компания украинский цена губернатор хорошавин шахта страна рубль помощь правительство савченко санкция взрыв проект человек становиться
Topic #2:
шахта взрыв засядько горняк донецк украина украинский помощь гуманитарный шахтер погибший пострадать колонна донецкий савченко авария погибать семья метан спасатель
Topic #3:
губернатор хорошавин сахалинский область сахалин александр президент глава задерживать савченко задержание следственный взятка обыск подозреваемый регион правительство дадаев дело арест
Topic #4:
савченко украинский украина российский надежда порошенко голодовка летчица президент гончаренко сизо письмо путин депутат решение заявлять граница журналист вгтрк пересечение
Topic #5:
цена нефть доллар рубль курс шахта торги биржа компания баррель россия рынок немцов

In [25]:
lsa_labels = [x.argmax() for x in lsa_matrix]
print_clusters(mart_news, lsa_labels)

Cluster: 0, count news: 161, titles:
	Смерть Немцова: «Минута славы» для бывших
	Из-за траурного марша движение в центре Москвы ограничат
	До поздней ночи в Москве несли цветы к месту гибели Бориса Немцова
	20 лет назад не стало Владислава Листьева
	Со дня гибели Владислава Листьева исполнилось 20 лет
	Гражданке Украины Дурицкой посоветовали оставаться в Москве
	Крупнейшие бизнесмены не пойдут на шествие памяти Бориса Немцова
	В центре Москвы завершился марш памяти Бориса Немцова
	Charlie Hedbo оскандалился с карикатурой про Донбасс
	Марш в память о Немцове: в Москве усилена безопасность
	В центре Москвы начался марш памяти Бориса Немцова
	Свидетель описал убийцу Бориса Немцова, составлен портрет
	За информацию об убийстве Немцова дадут три миллиона
	Траурный марш памяти Бориса Немцова: онлайн-трансляция
	Немцову хотят поставить памятник в Москве
	Убийство Немцова расследует специалист по делам националистов
	Убийство Немцова: полиция нашла владельца разыскиваемой "десятки"
	Спутница Н

# LDA

In [26]:
from sklearn.decomposition import LatentDirichletAllocation

In [27]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, tokenizer=lambda text: text.split(" "), stop_words=stopwords)
tf = tf_vectorizer.fit_transform(mart_content)

In [28]:
lda = LatentDirichletAllocation(n_topics=100, max_iter=100, learning_method='online',
                                learning_offset=50., )
lda_matrix = lda.fit_transform(tf)
print_topics(lda.components_, tfidf_vectorizer.get_feature_names(), 20)

Topic #0:
жмурки грабитель край дугужев корень беженец локоть неотделимый доставлять альтернатива легковушка единовременно бажова нигде воздвиженка австралопитек аргументированный женский заур барабошко
Topic #1:
велосезон берта неясно непострадавший жмурки монолог гридасова научаться индский донский нашуметь валютный балтика настраивать заслушиваться вулкан безумный бушевать гора модный
Topic #2:
берта гридасова авансовый барышня несчитывать нигерия аромат захар гусейнов надежность дипведомство безжалостный австрийский накачка новиков дифференцировать изнутри вольность несвойственный межпланетный
Topic #3:
данные возрастать запорожский гример йоханссон начинивать гамбургер аспект наворачиваться волочение невыездной выпуск лукашевич миниатюрный джеймс многократно еврозона булатов ландшафтный люцифер
Topic #4:
жмурки ненормативный домодедово кровавый артеменков неустанный дорогостоящий логика видимо дефендеры кулеб неликвидный лукашевич ленина леся берта биллинг заканчивать велосезон на

In [29]:
lda_labels = [x.argmax() for x in lda_matrix]
print_clusters(mart_news, lda_labels)

Cluster: 7, count news: 2, titles:
	Для Порошенко князь Владимир - основатель Украины, а для бандеровцев - русский захватчик
	Экс-глава ЦРУ признал, что передавал секретные данные любовнице
Cluster: 14, count news: 1, titles:
	Где летом в Москве будут самые длинные пробки
Cluster: 15, count news: 5, titles:
	Накануне весеннего равноденствия землян ждет полное солнечное затмение
	Авиакомпания организует из Мурманска спецрейс в солнечное затмение
	Женские радости: Печенье и сладости
	Зрителей предупредили об отключении телесигнала 8 марта
	Семь способов продлить жизнь цветам
Cluster: 19, count news: 1, titles:
	Полет ласки на дятле покорил пользователей Интернета
Cluster: 20, count news: 1, titles:
	Конькобежец Кулижников стал чемпионом мира в спринтерском многоборье
Cluster: 21, count news: 1, titles:
	Российские саночники завоевали серебро в эстафете Евро-2015
Cluster: 30, count news: 1, titles:
	Вся Британия обсуждает ролик, на котором из окна Букингемского дворца вылезает… голый мужч